In [11]:
from collision_check import check_collision
import numpy as np
cylinder1 = {
    "direct": np.array([1, 1, 1]),
    "center": np.array([1, 0, 0]),
    "radius": 1,
    "height": 1
}
cylinder2 = {
    "direct": np.array([0, 0, 1]),
    "center": np.array([-0.6, -0.6, 0]),
    "radius": 1,
    "height": 1
}
print(check_collision(cylinder1, cylinder2))

(True, {'parallel': False, 'coplane': False})


In [13]:
import plotly.graph_objects as go
import numpy as np

def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2 """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    if s == 0:
        return np.eye(3)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    return np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))

def draw_cylinder(center, radius, height, direction, color='blue'):
    objs = []  # Objects to add to the figure
    h = height
    direction = direction / np.linalg.norm(direction)  # Normalize direction vector
    # Rotation matrix to align circle normal to the cylinder direction
    rot_matrix = rotation_matrix_from_vectors(np.array([0, 0, 1]), direction)

    # Generate cylinder surface
    theta = np.linspace(0, 2 * np.pi, 100)
    steps = 100  # Number of steps along the cylinder's height
    for step in np.linspace(-0.5, 0.5, steps):
        circle_x = radius * np.cos(theta)
        circle_y = radius * np.sin(theta)
        circle_z = np.zeros_like(theta)  # Initially, circles are in the xy-plane
        circle_points = np.vstack((circle_x, circle_y, circle_z)).T
        circle_points = circle_points @ rot_matrix.T  # Apply rotation
        circle_points += center + direction * step * h  # Translate to position

        cylinder_surface = go.Scatter3d(x=circle_points[:, 0], y=circle_points[:, 1], z=circle_points[:, 2],
                                        mode='lines', line=dict(width=2, color=color),
                                        showlegend=False)
        objs.append(cylinder_surface)
    return objs



cylinder1_vis = draw_cylinder(cylinder1["center"], cylinder1["radius"], cylinder1["height"], cylinder1["direct"], color='red')
cylinder2_vis = draw_cylinder(cylinder2["center"], cylinder2["radius"], cylinder2["height"], cylinder2["direct"])
fig = go.Figure(data = [*cylinder1_vis, *cylinder2_vis])
# Update layout
fig.update_layout(scene=dict(aspectmode='data'), title="Cylinders Visualization with Plotly")
fig.show()

